# Interacting with [CodeGen](https://github.com/salesforce/CodeGen/)

- Originally from afiaka87
- Orginized Colab by mega b#6696
- Modified by Penguin-jpg

In [ ]:
#@markdown # Show gpu information
from IPython.display import clear_output
from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("500")')

!nvidia-smi

Thu Apr  7 02:25:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@markdown # Install requirements
!git clone https://github.com/salesforce/CodeGen
%cd CodeGen
!pip install --upgrade pip setuptools
!pip install -r requirements.txt

Cloning into 'CodeGen'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 99 (delta 45), reused 58 (delta 16), pack-reused 0
Unpacking objects: 100% (99/99), done.
/content/CodeGen
     |████████████████████████████████| 2.1 MB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 30.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 req

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 20.5 MB/s eta 0:00:01tcmalloc: large alloc 2041348096 bytes == 0x564b4b6a4000 @  0x7f53798e61e7 0x564b4886f407 0x564b4883917c 0x564b4891947a 0x564b4883bf9d 0x564b4892dd4d 0x564b488afec8 0x564b488aaa2e 0x564b4883d88a 0x564b488abb4f 0x564b488aaa2e 0x564b4883d88a 0x564b488abb4f 0x564b488aaa2e 0x564b4883d88a 0x564b488abb4f 0x564b488aaa2e 0x564b4883d88a 0x564b488abb4f 0x564b488aaa2e 0x564b4883d88a 0x564b488abb4f 0x564b4883d7aa 0x564b488abb4f 0x564b488aaa2e 0x564b4883d88a 0x564b488ac719 0x564b488aaa2e 0x564b4883d88a 0x564b488abb4f 0x564b488aaa2e
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 21.0 MB/s eta 0:00:01tcmalloc: large alloc 2551685120 bytes == 0x564bc516c000 @  0x7f53798e7615 0x564b4883917c 0x564b4891947a 0x564b4883bf9d 0x564b4892dd4d 0x564b488afec8 0x564b488aaa2e 0x564b4883d88a 0x564b488abb4f 0x564b488aaa2e 0x564b4883d88a 0x564b488abb4f 0x564b488a

In [ ]:
#@markdown # Load model and tokenizer
chosen_model = "codegen-350M-mono" #@param ["codegen-350M-nl", "codegen-350M-multi", "codegen-350M-mono", "codegen-2B-nl", "codegen-2B-multi", "codegen-2B-mono", "codegen-6B-nl", "codegen-6B-multi", "codegen-6B-mono", "codegen-16B-nl", "codegen-16B-multi", "codegen-16B-mono"]
fp16 = True #@param {type:"boolean"}

import os

if not os.path.exists(f'./checkpoints/{chosen_model}'):
  !wget -P checkpoints https://storage.googleapis.com/sfr-codegen-research/checkpoints/{chosen_model}.tar.gz && tar -xvf checkpoints/{chosen_model}.tar.gz -C checkpoints/


import torch
from jaxformer.hf.sample import truncate as do_truncate
from jaxformer.hf.sample import set_env, set_seed, print_time, create_model, create_custom_gpt2_tokenizer, create_tokenizer, sample

# (0) constants

models_nl = ['codegen-350M-nl', 'codegen-2B-nl', 'codegen-6B-nl', 'codegen-16B-nl']
models_pl = ['codegen-350M-multi', 'codegen-2B-multi', 'codegen-6B-multi', 'codegen-16B-multi', 'codegen-350M-mono', 'codegen-2B-mono', 'codegen-6B-mono', 'codegen-16B-mono']
models = models_nl + models_pl


# (2) preamble

set_env()

pad = 50256
device = torch.device('cuda:0')
ckpt = f'./checkpoints/{chosen_model}'

if device.type == "cpu":
  print()
  print("force full precision for cpu!!")
  print()
  fp16 = False


# (3) load

with print_time('loading parameters'):
  model = create_model(ckpt=ckpt, fp16=fp16).to(device)


with print_time('loading tokenizer'):
  if chosen_model in models_pl:
    tokenizer = create_custom_gpt2_tokenizer()
  else:
    tokenizer = create_tokenizer()
  tokenizer.padding_side = 'left'
  tokenizer.pad_token = pad

loading parameters
loading parameters took 17.28s
loading tokenizer
loading tokenizer took 1.16s


In [ ]:
#@markdown # Try out the model
rng_seed = 42 #@param {type:"integer"}
rng_deterministic = True #@param {type:"boolean"}
p = 0.95 #@param {type:"number"}
t = 0.2 #@param {type:"number"}
max_length = 128 #@param {type:"integer"}
batch_size = 1 #@param {type:"integer"}
context = "def hello_world():" #@param {type:"string"}

set_seed(rng_seed, deterministic=rng_deterministic)

# (4) sample

with print_time('sampling'):
  completion = sample(device=device, model=model, tokenizer=tokenizer, context=context, pad_token_id=pad, num_return_sequences=batch_size, temp=t, top_p=p, max_length_sample=max_length)[0]
  truncation = do_truncate(completion)

  print('=' * 100)
  print(completion)
  print('=' * 100)
  print(context+truncation)
  print('=' * 100)
    

# !python -m jaxformer.hf.sample --model $chosen_model \
#                  --rng-seed $rng_seed \
#                  --p $p \
#                  --t $t \
#                  --max-length $max_length \
#                  --batch-size $batch_size \
#                  --context '$context'
    

sampling

    print("Hello World")

hello_world()

#
def hello_world():
    print("Hello World")

hello_world()


sampling took 1.28s
